In [1]:
# importing libraries
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read the movie Dataset
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
    'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
    'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')

In [3]:
# Select only needed Item
movie_content=items[['movie id','movie title','Action', 'Adventure',
    'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
    'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]

In [4]:
# rename the movie id column name
movie_content.rename(columns={'movie id': 'movie_id'},inplace=True)

In [5]:
movie_content.head(3)

,movie_id,movie title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [6]:
#read the ratings dataset
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
user_item = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

user_item

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [7]:
user_item.isnull().sum()

user_id           0
movie_id          0
rating            0
unix_timestamp    0
dtype: int64

In [8]:
# Cross checking
user_item[user_item['user_id']==1][['movie_id','rating']]

,movie_id,rating
202,61,4
305,189,3
333,33,4
334,160,4
478,20,4
...,...,...
92049,28,4
92487,172,5
94019,122,3
96699,152,5


In [9]:
# Group by movie_id to count how many times each movie was watched
popularity = user_item.groupby('movie_id').agg(
    watch_count=('movie_id', 'count'),  # Count how many interactions
    avg_rating=('rating', 'mean')       # (Optional) Calculate average rating
).reset_index()
popularity

,movie_id,watch_count,avg_rating
0,1,452,3.878319
1,2,131,3.206107
2,3,90,3.033333
3,4,209,3.550239
4,5,86,3.302326
...,...,...,...
1677,1678,1,1.000000
1678,1679,1,3.000000
1679,1680,1,2.000000
1680,1681,1,3.000000


In [10]:
# Sort movies by watch_count in descending order
popularity = popularity.sort_values(by='watch_count', ascending=False).head(5)
popularity

,movie_id,watch_count,avg_rating
49,50,583,4.358491
257,258,509,3.803536
99,100,508,4.155512
180,181,507,4.007890
293,294,485,3.156701


In [11]:
print(popularity.columns)

Index(['movie_id', 'watch_count', 'avg_rating'], dtype='object')


In [12]:
# Merge popularity with movie dataset
merged = popularity.merge(movie_content, on='movie_id', how='left')
merged

,movie_id,watch_count,avg_rating,movie title,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,50,583,4.358491,Star Wars (1977),1,1,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
1,258,509,3.803536,Contact (1997),0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,100,508,4.155512,Fargo (1996),0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,181,507,4.007890,Return of the Jedi (1983),1,1,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
4,294,485,3.156701,Liar Liar (1997),0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Melt category columns into one 'category' column
category = items[['Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
    'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]

melted = merged.melt(id_vars=['movie_id', 'movie title', 'watch_count'], 
                     value_vars=category, 
                     var_name='category')#,                     value_name='flag')
melted

,movie_id,movie title,watch_count,category,value
0,50,Star Wars (1977),583,Action,1
1,258,Contact (1997),509,Action,0
2,100,Fargo (1996),508,Action,0
3,181,Return of the Jedi (1983),507,Action,1
4,294,Liar Liar (1997),485,Action,0
...,...,...,...,...,...
85,50,Star Wars (1977),583,Western,0
86,258,Contact (1997),509,Western,0
87,100,Fargo (1996),508,Western,0
88,181,Return of the Jedi (1983),507,Western,0


In [14]:
melted.drop(columns=['value'],inplace=True)
melted

,movie_id,movie title,watch_count,category
0,50,Star Wars (1977),583,Action
1,258,Contact (1997),509,Action
2,100,Fargo (1996),508,Action
3,181,Return of the Jedi (1983),507,Action
4,294,Liar Liar (1997),485,Action
...,...,...,...,...
85,50,Star Wars (1977),583,Western
86,258,Contact (1997),509,Western
87,100,Fargo (1996),508,Western
88,181,Return of the Jedi (1983),507,Western


In [15]:
melted['category'].value_counts()

category
Action         5
Adventure      5
Animation      5
Children's     5
Comedy         5
Crime          5
Documentary    5
Drama          5
Fantasy        5
Film-Noir      5
Horror         5
Musical        5
Mystery        5
Romance        5
Sci-Fi         5
Thriller       5
War            5
Western        5
Name: count, dtype: int64

In [16]:
# Pick top 5 per category
top5_per_category = melted.groupby('category').head(5)

In [17]:
top5_per_category

,movie_id,movie title,watch_count,category
0,50,Star Wars (1977),583,Action
1,258,Contact (1997),509,Action
2,100,Fargo (1996),508,Action
3,181,Return of the Jedi (1983),507,Action
4,294,Liar Liar (1997),485,Action
...,...,...,...,...
85,50,Star Wars (1977),583,Western
86,258,Contact (1997),509,Western
87,100,Fargo (1996),508,Western
88,181,Return of the Jedi (1983),507,Western


In [18]:
top5_per_category = melted.sort_values(['category', 'watch_count'], ascending=[True, False]).groupby('category').head(5)

In [19]:
top5_per_category

,movie_id,movie title,watch_count,category
0,50,Star Wars (1977),583,Action
1,258,Contact (1997),509,Action
2,100,Fargo (1996),508,Action
3,181,Return of the Jedi (1983),507,Action
4,294,Liar Liar (1997),485,Action
...,...,...,...,...
85,50,Star Wars (1977),583,Western
86,258,Contact (1997),509,Western
87,100,Fargo (1996),508,Western
88,181,Return of the Jedi (1983),507,Western


In [20]:
top5_per_category.to_csv('Top 5 Categories Movie Lists.csv',index=False)

In [21]:
top5_per_category.to_csv('Top 5 Categories Movie Lists.pdf',index=False)